In [179]:
from os import urandom
import hashlib
import unicodedata
import hmac

passphrase = ""
salt = unicodedata.normalize("NFKD", "").encode("utf-8")
ENT = 128
format_string = "{0:0"+str(ENT)+"b}"
ENTsie = ENT//8

CSsize = ENT//32

MS = (ENT+CSsize)//11

###################  TEST  ####################
random_number = urandom(ENTsie)
#random_number = b"\x00"*16
###############################################

CS = hashlib.sha256(random_number).digest()
CSstring = "{0:0256b}".format(int.from_bytes(CS,"big"),16)[:CSsize]

#This is a string binary representation
pre_binary_seed =  format_string.format(int.from_bytes(random_number,"big"),16) + CSstring


binary_seed_list = [pre_binary_seed[i*11:(i+1)*11] for i in range(MS) ]

words = []
with open("english_word.txt", "r", encoding="utf-8") as f:
    for w in f.readlines():
        words.append(w.strip())

mnemonic_words = []
for i in range(MS):
    mnemonic_words.append(words[int(binary_seed_list[i],2)])
    

salt = (unicodedata.normalize("NFKD","mnemonic" + passphrase)).encode("utf-8")


mnemonic = " ".join(mnemonic_words)
mnemonic = unicodedata.normalize("NFKD", mnemonic).encode("utf-8")
print(mnemonic)
    
bin_seed = hashlib.pbkdf2_hmac("sha512", mnemonic, salt, 2048)
 
print(mnemonic_words)
print(hex(int.from_bytes(bin_seed[:64],"big")))


b'antenna type flush across work dignity giggle lock giant session cruise together'
['antenna', 'type', 'flush', 'across', 'work', 'dignity', 'giggle', 'lock', 'giant', 'session', 'cruise', 'together']
0x1c1e4cee40cd2bfdfb2ea136523177ae4e60a3d64fa0c3f7e99e897fd6418c6ce008da4231d35a9a0e6a01d3dc64911a5bb71b747721ceec46fe6744a231c191


In [277]:
from os import urandom
import hashlib
import unicodedata
import hmac

class Mnemonic:
    
    def __init__(self,entropy = 128, words=None, random_number=None, passphrase="", seed=None):
        self.entropy = entropy
        self.words = words
        self.random_number = random_number
        self.passphrase = passphrase
        self.seed= seed
        self.MS = None
        
    def __repr__(self):
        return self.words.decode("utf-8") 
    
    @classmethod
    def generate_random(self,entropy = 128,  passphrase=""):
        
        ENT = entropy
        
        ENTsie = ENT//8
     

        random_number = urandom(ENTsie)
        
        return self.generate_from_number(number = random_number, entropy=entropy, passphrase = passphrase)
      
        
    @classmethod
    def generate_from_number(self,number = None, entropy = 128,  passphrase=""):
        
        ENT = entropy

        CSsize = ENT//32

        MS = (ENT+CSsize)//11
        
        return self.get_seed(ENT = entropy, random_number = number, passphrase = passphrase,MS=MS)
    
    @classmethod
    def recover_from_words(self,mnemonic_list = None, entropy = 128,  passphrase=""):
        
        return self.get_seed(mnemonic_list = mnemonic_list, passphrase = passphrase,MS=MS)
      
        
    @classmethod
    def get_seed(cls,ENT = 128, mnemonic_list=None, random_number=None,passphrase="",MS=None):
        
        words = []
        with open("english_word.txt", "r", encoding="utf-8") as f:
            for w in f.readlines():
                words.append(w.strip())

        if random_number is not None:

            if ENT not in [128,160,192,224,256]:
                return "not a valid entropy."                
            if isinstance(random_number,str):
                size = len(random_number)//2
                random_number = int(random_number,16).to_bytes(size,"big")
            elif isinstance(random_number,int):
                random_number.to_bytes(ENT//8,"big")
            elif isinstance(random_number,bytes):
                if len(random_number)*8 != ENT:
                    return "Length of random number must match entropy"
            else:
                return f"NOT a valid format for random number. Must be hex_string, int, or bytes. Not {type(random_number)}"


            CS = hashlib.sha256(random_number).digest()
            CSstring = "{0:0256b}".format(int.from_bytes(CS,"big"),16)[:CSsize]

            #This is a string binary representation
            format_string = "{0:0"+str(ENT)+"b}"
            pre_binary_seed =  format_string.format(int.from_bytes(random_number,"big"),16) + CSstring

            binary_seed_list = [pre_binary_seed[i*11:(i+1)*11] for i in range(MS) ]

            mnemonic_words = []
            for i in range(MS):
                mnemonic_words.append(words[int(binary_seed_list[i],2)])


        elif mnemonic_list is not None:
            if len(mnemonic_list) not in [12,15,18,21,24]:
                return "Not a valid amount of words. Must be 12, 15, 18, 21, or 24 word list."

            random_number_list = ["{0:016b}".format(words.index(x))[-11:] for x in mnemonic_list]
            random_number = "".join(random_number_list)
            size = len(random_number)//2
            random_number = int(random_number[:-(size//32)],2).to_bytes(size,"big")
            mnemonic_words = mnemonic_list

        salt = (unicodedata.normalize("NFKD","mnemonic" + passphrase)).encode("utf-8")

        mnemonic = " ".join(mnemonic_words)
        mnemonic = unicodedata.normalize("NFKD", mnemonic).encode("utf-8")

        seed = hashlib.pbkdf2_hmac("sha512", mnemonic, salt, 2048)

        return cls( entropy = ENT, words=mnemonic, random_number=random_number, 
                   passphrase=passphrase,seed=seed)




In [278]:
int("ff",16)

255

In [279]:
test = Mnemonic.generate_random(128)

In [280]:
test.words

b'album because claim eternal great speak wash limb tattoo physical grain similar'

In [281]:
test

album because claim eternal great speak wash limb tattoo physical grain similar

In [282]:
hex(int.from_bytes(test.seed,"big"))

'0xe9a48c9ea84f86dccb154ad7eec9f6829add5a0aee41108dd75020ac1b654345ecb788534f7c15fb0564575d6fc7d9e175f87bfbcf3d180ae1ec1cd57b6e586'

swear law nasty tell slice rail quarter desk into swing enrich pulse

'0x6a3b6e666d5b15c2590edbcb093aafb75fbbee5aa092fa54dabf48d195ce2140e193a0242c09358d6538949ddd73ab3c3f5d04520a3af1aa3439ef03c34203f8'

In [283]:
test2 = Mnemonic.recover_from_words(["swear", "law" ,"nasty" ,"tell", "slice" ,"rail" ,"quarter", "desk" ,"into", "swing", "enrich", "pulse"])

In [284]:
hex(int.from_bytes(test2.seed,"big"))

'0x6a3b6e666d5b15c2590edbcb093aafb75fbbee5aa092fa54dabf48d195ce2140e193a0242c09358d6538949ddd73ab3c3f5d04520a3af1aa3439ef03c34203f8'